# Resolution of the linear system $\langle \nabla P(x), v_i(x) \rangle = 0$ where the $v_i$ are our $d$ polynomial vector fields (that means that all the coordinates of $v_i$ are polynomials with indeterminates $x_1, \cdots, x_D$).

In [ ]:
run create_generic_polynomial.ipynb

In [ ]:
import numpy as np

get_coef_equations(P, Vector_fields):  

Parameters: 
- P: polynomial with $D$ indeterminates $x_1, \cdots, x_D$ and with indeterminate coefficients $a_1, \cdots, a_m$, 
- Vector_fields: np.array of size (D, d) whose columns correspond to the vector fields (e.g Vector_fields = $(\nabla \phi_1, \cdots, \nabla \phi_d)$). Each element of the array is a polynomial with $D$ indeterminates $x_1, \cdots, x_D$.

This function returns the list of the linear equations in the coefficients $a_i$, that the $a_i$ must satisfy if $\langle \nabla P(x), v_i (x)\rangle = 0$ for $ i = 1, \cdots , d$.

To do this, we calculate the system $\{\langle \nabla P(x), v_i (x) \rangle\}_{i}$ which is made up of $d$ linear equations in the coefs $a_i$ and polynomial in the $x_i$. Since we are looking for $P$ such that $\langle \nabla P(x), v_i (x)\rangle = 0$, we know that the coefficients (which are linear equations in the $a_i$) of the $d$ polynomials obtained are zero.

In [ ]:
def get_coef_equations(P, Vector_fields): 
    #  calculate the system  <gradP, Vector_fields> (d linear equations in the coef a_i, polynomial in the x_i)
    polynomial_system = P.gradient() @ Vector_fields 
    
    # calculate the coefficients of these d polynomials (linear in the a_i)
    list_coef_equations = []
    for i in range(len(polynomial_system)):
        list_coef_equations.extend(polynomial_system[i].coefficients())
        
    return list_coef_equations

In [ ]:
def build_matrix(k, Vector_fields, general = False):
    D = np.shape(Vector_fields)[0]
    if general == False:
        R, Ring, coef, variables_list, coefficients_list = ring(k, D)
        P = create_generic_homogeneous_polynomial(k, D)
    if general == True:
        R, Ring, coef, variables_list, coefficients_list = ring(k, D, general=True)
        # caution: coefficients_list is a list of lists in that case
        P = create_generic_polynomial(k, D)

    eq = get_coef_equations(P, Vector_fields)
    N, M = len(eq), len(coef)
    coefficient_matrix = np.empty((N, M))
    for i in range(N):
        degree_list = list(eq[i].degrees())
        non_zero_coefficients = list(eq[i].coefficients())
        u = 0
        for j in range(M):
            if degree_list[j] != 0:
                degree_list[j] = non_zero_coefficients[u]
                u += 1
        coefficient_matrix[i, :] = degree_list
    return M, coefficient_matrix.T, coef, coefficients_list, variables_list

In [5]:
def solve(k, Vector_fields, get_list_of_all_conserved_polynomial= False,  general = False):

    M, L, coef, coefficients_list, indeterminates_list = build_matrix(k,Vector_fields, general)
    R = PolynomialRing(RR, indeterminates_list)
    Mat = matrix(RR, L)
    ker = Mat.kernel()
    ker_basis = ker.basis()
    
    # next we reconstruct all polynomials associated with each vector in the kernel basis
    kernel_dim = len(ker_basis)
    P = []
    grad_P = []
    if kernel_dim == 0:
        return 0
    if general == False:
        for i in range(kernel_dim):
            Q = 0
            for j in range(len(coefficients_list)):
                Q += ker_basis[i][j]*R.monomial(*coefficients_list[j])
            P.append(Q)
            grad_P.append(Q.gradient())
        grad_P = np.array(grad_P).T
    if general == True:
        for i in range(kernel_dim):
            Q = 0
            m = 0
            for j in range(len(coefficients_list)):
                for k in range(len(coefficients_list[j])):
                    Q += ker_basis[i][k + m]*R.monomial(*coefficients_list[j][k])
                m += len(coefficients_list[j])
            P.append(Q)
            grad_P.append(Q.gradient())
        grad_P = np.array(grad_P).T

    # then we evaluate at a point and compute the rank
    u, v = np.shape(grad_P)
    evaluation = np.empty((u, v))
    D = np.shape(Vector_fields)[0]
    value = np.random.rand((D))
    dic = {}
    for k in range(D):
        dic[f"x{k+1}"] = value[k]
    for i in range(u):
        for j in range(v):
            evaluation[i, j] = grad_P[i][j](**dic)
    rank = np.linalg.matrix_rank(evaluation)
    
    #print("kernel dimension", M - Mat.rank())
    #print("number of independent polynomial conserved functions", rank)
    
    if get_list_of_all_conserved_polynomial == False:
        return rank
    else:
        return P, rank